# 📘 YAML → RST 自动化生成 Pipeline
本 Notebook 演示如何将结构化 YAML 文件转换为 reStructuredText (.rst)，以供 Sphinx 使用。

In [ ]:
import yaml, os

# === 1. 读取 YAML ===
with open('all_commands.yaml', 'r', encoding='utf-8') as f:
    data = yaml.safe_load(f)
commands = data['commands']

# === 2. 生成 RST ===
def generate_rst(cmd):
    rst = []
    rst.append(f"{cmd['command']}\n{'=' * len(cmd['command'])}\n")
    rst.append(f"**命令标题**：{cmd['title']}\n")
    rst.append(f"**命令类型**：{', '.join(cmd['type'])}\n\n")
    rst.append("命令格式::\n")
    for fmt in cmd['formats']:
        rst.append(f"   {fmt}")
    rst.append("")
    rst.append("参数说明\n--------\n")
    rst.append(".. list-table::\n   :header-rows: 1\n   :widths: 15 30 40\n")
    rst.append("   * - 参数名\n     - 描述\n     - 取值\n")
    for p in cmd.get('parameters', []):
        values = '\\n'.join([f"{k}: {v}" for k, v in p['values'].items()])
        rst.append(f"   * - {p['name']}\n     - {p['desc']}\n     - {values}")
    rst.append("")
    rst.append("示例\n----\n")
    for ex in cmd.get('examples', []):
        rst.append(f".. code-block:: none\n\n   {ex['cmd']}\n   {ex['resp']}\n")
    rst.append("")
    rst.append(f"**功能描述**：{cmd.get('description','')}\n\n")
    if cmd.get('notes'):
        rst.append(f"**注意**：{cmd['notes']}\n")
    return '\n'.join(rst)

# === 3. 输出 RST 文件 ===
os.makedirs('rst_output', exist_ok=True)
for cmd in commands:
    rst_text = generate_rst(cmd)
    with open(f"rst_output/{cmd['command']}.rst", 'w', encoding='utf-8') as f:
        f.write(rst_text)
print('✅ 已生成 rst_output/*.rst')

# === 4. 生成 index.rst ===
index = ['AT 命令手册\n==================\n', '.. toctree::\n   :maxdepth: 1\n']
for cmd in commands:
    index.append(f"   {cmd['command']}")
with open('rst_output/index.rst', 'w', encoding='utf-8') as f:
    f.write('\n'.join(index))
print('✅ 已生成 rst_output/index.rst')